In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt


In [8]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [9]:
import json
import requests
import subprocess
import time
import uuid

from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)

from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Hide additional warnings in the notebook
import warnings

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [10]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [12]:
# Define an exercise catalog lookup tool
def get_exercise_info(exercise_name: str) -> str:
    """Get exercise information for a given workout.

    Args:
        exercise_name: Name of the exercise (e.g., "push up", "squat")

    Returns:
        exercise information as a string
    """

    exercise_seed = {
        "push up": "Push-up | Muscles: Chest, Triceps | Difficulty: Beginner | Equipment: None | Reps: 10-20",
        "squat": "Squat | Muscles: Legs, Core | Difficulty: Beginner | Equipment: None | Reps: 12-20",
        "plank": "Plank | Muscles: Core | Difficulty: Beginner | Equipment: None | Duration: 30-90s",
        "burpee": "Burpee | Muscles: Full Body | Difficulty: Intermediate | Equipment: None | Reps: 8-15",
        "deadlift": "Deadlift | Muscles: Back, Legs | Difficulty: Advanced | Equipment: Barbell | Reps: 5-8",
        "jumping jack": "Jumping Jack | Muscles: Full Body | Difficulty: Beginner | Equipment: None | Reps: 30-60",
        "mountain climber": "Mountain Climber | Muscles: Core, Cardio | Difficulty: Intermediate | Equipment: None | Reps: 20-40",
    }

    exercise_lower = exercise_name.lower().strip()

    if exercise_lower in exercise_seed:
        return f"Exercise: {exercise_seed[exercise_lower]}"
    else:
        available = ", ".join([name.title() for name in exercise_seed.keys()])
        return (
            f"Sorry, I don't have information for '{exercise_name}'. "
            f"Available exercises: {available}"
        )


# Create the Exercise Catalog Agent
exercise_agent = LlmAgent(
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    name="exercise_agent",
    description="Knowledgeable fitness trainer agent that provides exercise information.",
    instruction="""
You are a certified fitness trainer with years of experience.
When the user asks about an exercise, use the get_exercise_info tool to fetch details.
Be friendly, motivating, and helpful. Provide clear, structured exercise information.
""",
    tools=[get_exercise_info],
)

print("✅ Exercise Agent created successfully!")
print("   Model: gemini-2.5-flash-lite")
print("   Tool: get_exercise_info()")
print("   Ready to be exposed via A2A...")


✅ Exercise Agent created successfully!
   Model: gemini-2.5-flash-lite
   Tool: get_exercise_info()
   Ready to be exposed via A2A...


In [13]:
# Convert the product catalog agent to an A2A-compatible application
# This creates a FastAPI/Starlette app that:
#   1. Serves the agent at the A2A protocol endpoints
#   2. Provides an auto-generated agent card
#   3. Handles A2A communication protocol
exercise_a2a_app = to_a2a(
    exercise_agent, port=8001  # Port where this agent will be served
)

print("✅ Exercise Agent is now A2A-compatible!")
print("   Agent will be served at: http://localhost:8001")
print("   Agent card will be at: http://localhost:8001/.well-known/agent-card.json")
print("   Ready to start the server...")

✅ Exercise Agent is now A2A-compatible!
   Agent will be served at: http://localhost:8001
   Agent card will be at: http://localhost:8001/.well-known/agent-card.json
   Ready to start the server...


In [15]:
# First, let's save the product catalog agent to a file that uvicorn can import
exercise_agent_code = '''
import os
from google.adk.agents import LlmAgent
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)
# Define an exercise catalog lookup tool
def get_exercise_info(exercise_name: str) -> str:
    """Get exercise information for a given workout.

    Args:
        exercise_name: Name of the exercise (e.g., "push up", "squat")

    Returns:
        exercise information as a string
    """

    exercise_seed = {
        "push up": "Push-up | Muscles: Chest, Triceps | Difficulty: Beginner | Equipment: None | Reps: 10-20",
        "squat": "Squat | Muscles: Legs, Core | Difficulty: Beginner | Equipment: None | Reps: 12-20",
        "plank": "Plank | Muscles: Core | Difficulty: Beginner | Equipment: None | Duration: 30-90s",
        "burpee": "Burpee | Muscles: Full Body | Difficulty: Intermediate | Equipment: None | Reps: 8-15",
        "deadlift": "Deadlift | Muscles: Back, Legs | Difficulty: Advanced | Equipment: Barbell | Reps: 5-8",
        "jumping jack": "Jumping Jack | Muscles: Full Body | Difficulty: Beginner | Equipment: None | Reps: 30-60",
        "mountain climber": "Mountain Climber | Muscles: Core, Cardio | Difficulty: Intermediate | Equipment: None | Reps: 20-40",
    }

    exercise_lower = exercise_name.lower().strip()

    if exercise_lower in exercise_seed:
        return f"Exercise: {exercise_seed[exercise_lower]}"
    else:
        available = ", ".join([name.title() for name in exercise_seed.keys()])
        return (
            f"Sorry, I don't have information for '{exercise_name}'. "
            f"Available exercises: {available}"
        )


# Create the Exercise Catalog Agent
exercise_agent = LlmAgent(
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    name="exercise_agent",
    description="Knowledgeable fitness trainer agent that provides exercise information.",
    instruction="""
You are a certified fitness trainer with years of experience.
When the user asks about an exercise, use the get_exercise_info tool to fetch details.
Be friendly, motivating, and helpful. Provide clear, structured exercise information.
""",
    tools=[get_exercise_info],
)


# Create the A2A app
app = to_a2a(exercise_agent, port=8001)
'''

# Write the product catalog agent to a temporary file
with open("/tmp/exercise_server.py", "w") as f:
    f.write(exercise_agent_code)

print("📝 Exercise agent code saved to /tmp/product_catalog_server.py")

# Start uvicorn server in background
# Note: We redirect output to avoid cluttering the notebook
server_process = subprocess.Popen(
    [
        "uvicorn",
        "exercise_server:app",  # Module:app format
        "--host",
        "localhost",
        "--port",
        "8001",
    ],
    cwd="/tmp",  # Run from /tmp where the file is
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env={**os.environ},  # Pass environment variables (including GOOGLE_API_KEY)
)

print("🚀 Starting exercise Agent server...")
print("   Waiting for server to be ready...")

# Wait for server to start (poll until it responds)
max_attempts = 30
for attempt in range(max_attempts):
    try:
        response = requests.get(
            "http://localhost:8001/.well-known/agent-card.json", timeout=1
        )
        if response.status_code == 200:
            print(f"\n✅ Exercise Agent server is running!")
            print(f"   Server URL: http://localhost:8001")
            print(f"   Agent card: http://localhost:8001/.well-known/agent-card.json")
            break
    except requests.exceptions.RequestException:
        time.sleep(5)
        print(".", end="", flush=True)
else:
    print("\n⚠️  Server may not be ready yet. Check manually if needed.")

# Store the process so we can stop it later
globals()["exercise_server_process"] = server_process

📝 Exercise agent code saved to /tmp/product_catalog_server.py
🚀 Starting exercise Agent server...
   Waiting for server to be ready...
....
✅ Exercise Agent server is running!
   Server URL: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json


In [16]:
# Fetch the agent card from the running server
try:
    response = requests.get(
        "http://localhost:8001/.well-known/agent-card.json", timeout=5
    )

    if response.status_code == 200:
        agent_card = response.json()
        print("📋 Exercise Agent Card:")
        print(json.dumps(agent_card, indent=2))

        print("\n✨ Key Information:")
        print(f"   Name: {agent_card.get('name')}")
        print(f"   Description: {agent_card.get('description')}")
        print(f"   URL: {agent_card.get('url')}")
        print(f"   Skills: {len(agent_card.get('skills', []))} capabilities exposed")
    else:
        print(f"❌ Failed to fetch agent card: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Error fetching agent card: {e}")
    print("   Make sure the Exercise Agent server is running (previous cell)")

📋 Exercise Agent Card:
{
  "capabilities": {},
  "defaultInputModes": [
    "text/plain"
  ],
  "defaultOutputModes": [
    "text/plain"
  ],
  "description": "Knowledgeable fitness trainer agent that provides exercise information.",
  "name": "exercise_agent",
  "preferredTransport": "JSONRPC",
  "protocolVersion": "0.3.0",
  "skills": [
    {
      "description": "Knowledgeable fitness trainer agent that provides exercise information. \nI am a certified fitness trainer with years of experience.\nWhen the user asks about an exercise, use the get_exercise_info tool to fetch details.\nBe friendly, motivating, and helpful. Provide clear, structured exercise information.\n",
      "id": "exercise_agent",
      "name": "model",
      "tags": [
        "llm"
      ]
    },
    {
      "description": "Get exercise information for a given workout.\n\nArgs:\n    exercise_name: Name of the exercise (e.g., \"push up\", \"squat\")\n\nReturns:\n    exercise information as a string",
      "id": "e

In [17]:
# Create a RemoteA2aAgent that connects to our Product Catalog Agent
# This acts as a client-side proxy - the Customer Support Agent can use it like a local agent
remote_exercise_agent = RemoteA2aAgent(
    name="exercise_agent",
    description="Remote exercise agent from specalist that provides exercise information.",
    # Point to the agent card URL - this is where the A2A protocol metadata lives
    agent_card=f"http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}",
)

print("✅ Remote exercise Agent proxy created!")
print(f"   Connected to: http://localhost:8001")
print(f"   Agent card: http://localhost:8001{AGENT_CARD_WELL_KNOWN_PATH}")
print("   The Customer Support Agent can now use this like a local sub-agent!")

✅ Remote exercise Agent proxy created!
   Connected to: http://localhost:8001
   Agent card: http://localhost:8001/.well-known/agent-card.json
   The Customer Support Agent can now use this like a local sub-agent!


In [18]:
# Create the User Agent that communicates with the exercise catalog agent
user_agent = LlmAgent(
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    name="user_agent",
    description="Front-facing fitness assistant for exercise guidance.",
    instruction="""
You are a friendly, motivating, and knowledgeable fitness assistant.

When the user asks about an exercise:
1. ALWAYS use the exercise_catalog_agent sub-agent to look up exercise information.
2. Return details such as muscles trained, difficulty, equipment needed, sets/reps or duration.
3. Explain the exercise safely and clearly.
4. Provide supportive and positive fitness advice.

Be warm, encouraging, and professional.
""",
    sub_agents=[exercise_agent],   # Add your exercise catalog sub-agent
)

print("✅ User Fitness Agent created!")
print("   Model: gemini-2.5-flash-lite")
print("   Sub-Agent: exercise_agent (via A2A)")
print("   Ready to assist users!")


✅ User Fitness Agent created!
   Model: gemini-2.5-flash-lite
   Sub-Agent: exercise_agent (via A2A)
   Ready to assist users!


In [19]:
async def test_a2a_communication(user_query: str):
    """
    Test the A2A communication between User Agent and Exercise Catalog Agent.

    This function:
    1. Creates a new session for this conversation
    2. Sends the query to the User Agent
    3. User Agent communicates with Exercise Catalog Agent via A2A
    4. Displays the response

    Args:
        user_query: The question to ask the User Agent
    """
    # Setup session management (required by ADK)
    session_service = InMemorySessionService()

    # Session identifiers
    app_name = "fitness_app"
    user_id = "demo_user"
    # Use unique session ID for each test to avoid conflicts
    session_id = f"demo_session_{uuid.uuid4().hex[:8]}"

    # CRITICAL: Create session BEFORE running agent (synchronous, not async!)
    # This pattern matches the deployment notebook exactly
    session = await session_service.create_session(
        app_name=app_name, user_id=user_id, session_id=session_id
    )

    # Create runner for the User Agent
    # The runner manages the agent execution and session state
    runner = Runner(
        agent=user_agent, app_name=app_name, session_service=session_service
    )

    # Create the user message
    # This follows the same pattern as the deployment notebook
    test_content = types.Content(parts=[types.Part(text=user_query)])

    # Display query
    print(f"\n👤 Customer: {user_query}")
    print(f"\n🎧 User Agent response:")
    print("-" * 60)

    # Run the agent asynchronously (handles streaming responses and A2A communication)
    async for event in runner.run_async(
        user_id=user_id, session_id=session_id, new_message=test_content
    ):
        # Print final response only (skip intermediate events)
        if event.is_final_response() and event.content:
            for part in event.content.parts:
                if hasattr(part, "text"):
                    print(part.text)

    print("-" * 60)


# Run the test
print("🧪 Testing A2A Communication...\n")
await test_a2a_communication("Can you tell me about Push Up? How many reps should I do?")


🧪 Testing A2A Communication...


👤 Customer: Can you tell me about Push Up? How many reps should I do?

🎧 User Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


The push-up is a great compound exercise that primarily targets your chest, shoulders, and triceps. It's a fantastic bodyweight exercise that requires no equipment, making it accessible anywhere!

For beginners, aiming for 10-20 reps is a good starting point. Focus on maintaining good form:

*   **Starting Position:** Place your hands slightly wider than shoulder-width apart on the floor, fingers pointing forward. Extend your legs back, balancing on your toes. Your body should form a straight line from your head to your heels.
*   **Lowering Phase:** Keeping your core engaged and body straight, lower yourself towards the floor by bending your elbows. Aim to get your chest as close to the floor as possible.
*   **Pushing Phase:** Push back up to the starting position, extending your arms fully but without locking your elbows.

**Important Tips:**

*   **Form over Quantity:** It's much better to do fewer push-ups with perfect form than many with poor form. This prevents injuries and ensu

In [21]:
await test_a2a_communication(
    "I was confused between which exercise to do ,cause which one is best push ups or squats"
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: I was confused between which exercise to do ,cause which one is best push ups or squats

🎧 User Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


Hey there! It's awesome you're looking to get into fitness. Both push-ups and squats are fantastic exercises, but they work different parts of your body. To give you the best advice, I need a little more information.

Could you tell me what your fitness goals are? For example, are you looking to:

*   **Build upper body strength?** (Push-ups are great for this!)
*   **Strengthen your legs and glutes?** (Squats are king here!)
*   **Improve overall body strength and endurance?** (Both are excellent!)
*   **Focus on a specific area?**

Once I know what you're aiming for, I can help you decide which exercise, or combination of exercises, would be best for you right now! 😊
------------------------------------------------------------


In [24]:
await test_a2a_communication(
    "What should be standard time for a plank?"
)

INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False



👤 Customer: What should be standard time for a plank?

🎧 User Agent response:
------------------------------------------------------------


INFO:google_adk.google.adk.models.google_llm:Response received from the model.
INFO:google_adk.google.adk.models.google_llm:Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
INFO:google_adk.google.adk.models.google_llm:Response received from the model.


A plank is a fantastic exercise for building core strength and endurance! 💪

The "standard" time for a plank can vary greatly depending on your current fitness level. For beginners, even holding a plank for 10-20 seconds with good form is a great start!

As you get stronger, you can gradually increase the hold time. Many people aim for 30 seconds to 1 minute as a solid benchmark. Advanced athletes might hold planks for several minutes!

Here's a quick guide:

*   **Beginner:** 10-30 seconds
*   **Intermediate:** 30-60 seconds
*   **Advanced:** 1-3 minutes or more!

**Key Tips for a Great Plank:**

1.  **Form over duration:** It's much better to hold a plank for a shorter time with perfect form than for a long time with poor form.
2.  **Engage your core:** Really squeeze your abdominal muscles as if you're bracing for a punch.
3.  **Keep your body straight:** Your body should form a straight line from your head to your heels. Avoid letting your hips sag or rise too high.
4.  **Don't hol